In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Import and rearrange data

In [2]:
# Load data
bkgd = pd.read_hdf("data/preprocessed/bkgd.h5", key="bkgd")
vlq = pd.read_hdf("data/preprocessed/vlq.h5", key="vlq")
X_train = pd.concat([bkgd, vlq])
del bkgd, vlq

In [3]:
# Train, test and validation sets
X_train, X_test, y_train, y_test = train_test_split(X_train.drop(["Label"], axis=1), X_train["Label"], 
                                                    test_size=1/3, random_state=56)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=60)

In [4]:
# Save train data
X_train.to_hdf("data/classifier/train.h5", key="X")
y_train.to_hdf("data/classifier/train.h5", key="y")

# Save validation data
X_val.to_hdf("data/classifier/validation.h5", key="X")
y_val.to_hdf("data/classifier/validation.h5", key="y")

# Save test data
X_test.to_hdf("data/classifier/test.h5", key="X")
y_test.to_hdf("data/classifier/test.h5", key="y")

In [5]:
# Get data samples
train_samples, val_samples, test_samples = X_train["Sample"], X_val["Sample"], X_test["Sample"]

# Get data weights
train_weights, val_weights, test_weights = X_train["gen_weights"], X_val["gen_weights"], X_test["gen_weights"]

# Remove sample and weight columns
X_train.drop(["Sample", "gen_weights"], axis=1, inplace=True)
X_val.drop(["Sample", "gen_weights"], axis=1, inplace=True)
X_test.drop(["Sample", "gen_weights"], axis=1, inplace=True)

# Calculate class weights
class_weights = {
    0: 1,
    1: len(y_train[y_train==0]) / len(y_train[y_train==1])
}

# Create model

In [7]:
def get_model(hidden_layers=[100, 100, 100], dropout=0.1, batch_norm=True, optimizer="Nadam"):
    """
    This function creates a keras model, given the desired hidden_layers, dropout rate
    and optimizer of choice
    
    hidden_layers -> [int]: size of each desired hidden layer
    dropout -> float: desired dropout rate
    optimizer -> string: optimizer you choose to utilize
    
    returns a keras model
    """
    
    
    # Generate model structure
    inputs = keras.Input(shape=(69,))
    bn = keras.layers.BatchNormalization()(inputs)
    drop = bn
    for i in range(len(hidden_layers)-1):
        fc = keras.layers.Dense(hidden_layers[i], activation='relu')(drop)
        if batch_norm:
            bn = keras.layers.BatchNormalization()(fc)
        else:
            bn = fc
        drop = keras.layers.Dropout(dropout)(bn, training=True)
    fc = keras.layers.Dense(hidden_layers[-1], activation='relu')(drop)
    outputs = keras.layers.Dense(1, activation='sigmoid')(fc)
    
    # Instanciate and compile model
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss="binary_crossentropy",
                  metrics=["accuracy", keras.metrics.AUC()])
    model.summary()
    
    return model

In [8]:
# Model hyperparameters
hidden_layers = [100, 100, 100]
batch_size = 1024
dropout = 0.15
batch_norm = True
optimizer = "Nadam"

In [9]:
# Model 
model = get_model(hidden_layers, dropout, batch_norm, optimizer)

# Model name
name = "Hidden:" + str(hidden_layers).replace("[","").replace("]","").replace(" ","") + "|"
name += f"BatchS:{batch_size}|Dropout:{dropout}|"
if batch_norm: name += "BatchNorm|"
name += optimizer

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 69)]              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 69)                276       
_________________________________________________________________
dense (Dense)                (None, 100)               7000      
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)              

In [10]:
name

'Hidden:100,100,100|BatchS:1024|Dropout:0.15|BatchNorm|Nadam'

# Callbacks

In [11]:
# Tensorboard
TB = keras.callbacks.TensorBoard("logs/" + name, write_images=True)

# Early Stopping
ES = keras.callbacks.EarlyStopping(monitor="val_loss", patience=20, mode="min")

# Model Checkpoint
MC = keras.callbacks.ModelCheckpoint("models/" + name + ".h5", save_best_only=True, monitor="val_loss",
                                     mode="min")

# Reduce LR on Plateau
LR = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, mode="min", 
                                       min_lr=1e-6)

# Training the Model

In [12]:
model.fit(X_train.values, y_train.values, batch_size=batch_size, epochs=500, callbacks=[TB, ES, MC, LR],
          validation_data=(X_val.values, y_val.values, val_weights.values), shuffle=True,
          sample_weight=train_weights.values, class_weight=class_weights)

Epoch 1/500
  1/270 [..............................] - ETA: 0s - loss: 4.1683e-04 - accuracy: 0.4990 - auc: 0.3435WARNING:tensorflow:From /home/gilbertocunha/.local/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
270/270 [==============================] - 14s 51ms/step - loss: 1.5492e-05 - accuracy: 0.8573 - auc: 0.9247 - val_loss: 2.0607e-06 - val_accuracy: 0.8526 - val_auc: 0.9572
Epoch 2/500
270/270 [==============================] - 2s 8ms/step - loss: 1.6634e-06 - accuracy: 0.8300 - auc: 0.9437 - val_loss: 1.1953e-06 - val_accuracy: 0.8498 - val_auc: 0.9502
Epoch 3/500
270/270 [==============================] - 2s 8ms/step - loss: 1.3808e-06 - accuracy: 0.8452 - auc: 0.9447 - val_loss: 8.2291e-07 - val_accuracy: 0.8055 - val_auc: 0.9233
Epoch 4/500
270/270 [==============================] 